In [1]:
import cv2
from ultralytics import YOLO
import os

In [14]:
def run_video_inference(model_path: str, input_video_path: str, output_directory: str) -> None:
    model = YOLO(model_path)
    print("Model loaded.")

    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        print("Error opening video file.")
        return

    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    video_duration = total_frames / fps  # in seconds

    base_name = os.path.basename(input_video_path)
    file_name, _ = os.path.splitext(base_name)
    output_video_path = os.path.join(output_directory, f"{file_name}_processed.mp4")

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (int(cap.get(3)), int(cap.get(4))))

    chews_count = 0
    previous_state = 'Open'  # Assuming it starts 'open', adjust as necessary based on actual data

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("No frames read, breaking loop.")
            break

        results = model(frame)

        current_state = 'Open'  # Default assumption
        for result in results:
            boxes = result.boxes
            if len(boxes) > 0:
                boxes = boxes[0]
            for box in boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                conf = box.conf[0]
                cls = int(box.cls[0])
                label = model.names[cls]

                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

                if label == 'Close':
                    current_state = 'Close'
                    break  # We only need the first 'closed' state per frame

        if current_state == 'Close' and previous_state == 'Open':
            chews_count += 1

        previous_state = current_state  # Update the state for the next frame
        out.write(frame)
        print("Frame written to file.")

    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print("Video processing completed.")

    chews_per_minute = (chews_count / video_duration) * 60 if video_duration > 0 else 0  # scaled for 60 seconds
    print(f"Chews count per minute: {int(chews_per_minute)}")

In [15]:
path_to_model = "best.pt"
input_video_path1 = "samples/rumination.mp4"
input_video_path2 = "samples/rumination2.mp4"

output_dir = "outputs"

In [16]:
run_video_inference(path_to_model, input_video_path1, output_dir)

Model loaded.

0: 640x384 (no detections), 3.6ms
Speed: 1.2ms preprocess, 3.6ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 384)
Frame written to file.

0: 640x384 (no detections), 3.7ms
Speed: 1.2ms preprocess, 3.7ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 384)
Frame written to file.

0: 640x384 (no detections), 3.5ms
Speed: 1.3ms preprocess, 3.5ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 384)
Frame written to file.

0: 640x384 (no detections), 3.5ms
Speed: 1.3ms preprocess, 3.5ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 384)
Frame written to file.

0: 640x384 (no detections), 3.9ms
Speed: 1.5ms preprocess, 3.9ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 384)
Frame written to file.

0: 640x384 (no detections), 3.7ms
Speed: 1.7ms preprocess, 3.7ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 384)
Frame written to file.

0: 640x384 (no detections), 4.1ms
Speed: 1.5ms preprocess, 4.

In [17]:
run_video_inference(path_to_model, input_video_path2, output_dir)

Model loaded.

0: 640x384 (no detections), 3.8ms
Speed: 1.2ms preprocess, 3.8ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 384)
Frame written to file.

0: 640x384 (no detections), 4.0ms
Speed: 2.0ms preprocess, 4.0ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 384)
Frame written to file.

0: 640x384 (no detections), 4.1ms
Speed: 1.4ms preprocess, 4.1ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 384)
Frame written to file.

0: 640x384 (no detections), 3.8ms
Speed: 2.6ms preprocess, 3.8ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 384)
Frame written to file.

0: 640x384 (no detections), 4.0ms
Speed: 1.8ms preprocess, 4.0ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 384)
Frame written to file.

0: 640x384 (no detections), 4.8ms
Speed: 1.8ms preprocess, 4.8ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 384)
Frame written to file.

0: 640x384 (no detections), 4.5ms
Speed: 2.0ms preprocess, 4.